In [1]:
%%time
import pandas as pd
import os, sys, time, json, re, string, datetime

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import OneHotEncoder, HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler, PCA, OneHotEncoderEstimator,StringIndexer

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.classification import GBTClassifier, MultilayerPerceptronClassifier

from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

3.6.10
CPU times: user 51.1 ms, sys: 18.8 ms, total: 69.9 ms
Wall time: 67.7 ms


In [2]:
%%time
sc

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


<SparkContext master=yarn appName=PySparkShell>

In [3]:
%%time
spark = SparkSession.builder \
        .appName("fakenews_enhanced_model") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "15g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "10g") \
        .getOrCreate()

CPU times: user 8.68 ms, sys: 97 µs, total: 8.78 ms
Wall time: 11.2 ms


In [4]:
%%time
fakenews_path="gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/"
fakenews_data_path = fakenews_path + "two_million_rows_news_cleaned_2018_02_13_pyspark.csv"
df_news = spark.read.format("com.databricks.spark.csv") \
                    .option("header", "true") \
                    .option("delimiter", '#') \
                    .load(fakenews_data_path)

#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.type != "")
df_news = df_news.filter(df_news.content != "")
df_news = df_news.filter(df_news.domain != "")
df_news = df_news.filter(df_news.title != "")
df_news = df_news.filter(df_news.authors != "")
df_news = df_news.dropDuplicates(['type', 'content', 'title', 'authors'])

CPU times: user 13.3 ms, sys: 777 µs, total: 14 ms
Wall time: 6.6 s


In [5]:
# only keep type and content
df_news = df_news.select('type', 'content', 'domain', 'title','authors')
split_authors_col = F.split(df_news['authors'], ', ')
df_news = df_news.withColumn('author1', split_authors_col.getItem(0))
df_news = df_news.withColumn('author2', split_authors_col.getItem(1))

# add binary label
df_news = df_news.withColumn("label", F.when(F.col("type") == 'fake', 1).otherwise(0))

#facing out of memory issue in google cluster for 5 instance * 15 GB, so sampling with 10K fake news and non-fake news
row_count = 10000
df_news_fake = df_news.filter(df_news.type == 'fake').limit(row_count)
df_news_nonfake = df_news.filter(df_news.type != 'fake').limit(row_count)
df_news = df_news_fake.union(df_news_nonfake)

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)
param_tuning = False

In [6]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 95 µs, sys: 13 µs, total: 108 µs
Wall time: 114 µs


In [7]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 71 µs, sys: 10 µs, total: 81 µs
Wall time: 85.1 µs


In [8]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.2 µs


In [9]:
%%time

def run_models(df_train, df_test):
    print("**********LogisticRegression**********")
    t = time.time()
    lr_model = LogisticRegression(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  family='binomial', 
                                  fitIntercept=True, 
                                  threshold=0.5, 
                                  standardization=False, 
                                  maxIter=200, 
                                  regParam=0.005, 
                                  elasticNetParam=0, 
                                  tol=1e-06, 
                                  aggregationDepth=2)

    lr_model = lr_model.fit(df_train)
    
    eval_model_perf(lr_model, df_test)
    
    print("time taken for LogisticRegression: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()

    # BELOW CODE IS USED ONLY FOR PARAM TUNNING
    # # grid search params tunning
    # paramGrid = ParamGridBuilder() \
    #     .addGrid(lr_model.regParam, [0.001, 0.005]) \
    #     .addGrid(lr_model.standardization, [False, True]) \
    #     .build()
    # # cross validator model
    # crossval = CrossValidator(estimator=lr_model, 
    # #                           evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
    #                           evaluator=BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="label", metricName="areaUnderROC"),
    #                           estimatorParamMaps=paramGrid,
    #                           numFolds=2, parallelism = 2,
    #                           seed=666)
    # # run it!
    # crossval_model = crossval.fit(df_train)
    # # evaluate performance
    # show_cv_results(crossval_model)
    # eval_model_perf(crossval_model, df_test)
    # selected_model = crossval_model.bestModel
    # print(selected_model.explainParams())
    print("**********DecisionTreeClassifier**********")
    dt_model = DecisionTreeClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction', 
                                      maxDepth=10, maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10,
                                      impurity='gini', 
                                      seed=666)

    dt_model = dt_model.fit(df_train)
    eval_model_perf(dt_model, df_test)
    print("time taken for DecisionTreeClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()
    
    print("**********RandomForestClassifier**********")
    rf_model = RandomForestClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction',
                                      maxDepth=10, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10, 
                                      impurity='gini', 
                                      numTrees=200, 
                                      featureSubsetStrategy='auto', 
                                      seed=666, 
                                      subsamplingRate=0.8)

    rf_model = rf_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for RandomForestClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()
    
    print("**********GBTClassifier**********")
    gbt_model = GBTClassifier(featuresCol='features', 
                             labelCol='label', 
                             maxIter=250)

    gbt_model = gbt_model.fit(df_train)
    eval_model_perf(gbt_model, df_test)
    print("time taken for GBTClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()    
    
    print("**********MultilayerPerceptronClassifier**********")
    mp_model = MultilayerPerceptronClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction', 
                                              layers=[4, 5, 4, 3],  
                                              maxIter=100, 
                                              blockSize=128, 
                                              seed=1234)

    mp_model = mp_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for MultilayerPerceptronClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()  

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 12.9 µs


In [10]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        RegexTokenizer(inputCol="title", outputCol="all_title_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_title_words", outputCol="title_words"),
        CountVectorizer(inputCol="title_words", outputCol="title_tf_features", vocabSize=100),
        IDF(inputCol="title_tf_features", outputCol="title_tfidf_features"),
        PCA(inputCol="title_tfidf_features", outputCol="title_pca_features", k=100),        

        #StringIndexer(inputCol="domain", outputCol="domain_indexed", handleInvalid='keep'),
        #OneHotEncoder(inputCol="domain_indexed", outputCol="domain_feature"),
        
        StringIndexer(inputCol="author1", outputCol="author1_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="author1_indexed", outputCol="author1_feature"),
        
        StringIndexer(inputCol="author2", outputCol="author2_indexed", handleInvalid='keep'),        
        OneHotEncoder(inputCol="author2_indexed", outputCol="author2_feature"),
        
        VectorAssembler(inputCols=["pca_features", "title_pca_features", 
                                   "title_tfidf_features", 
                                   "content_features", "word_features",
                                   "author1_feature", "author2_feature"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        
        #StringIndexer(inputCol="domain", outputCol="domain_indexed", handleInvalid='keep'),
        #OneHotEncoder(inputCol="domain_indexed", outputCol="domain_feature"),
        
        StringIndexer(inputCol="author1", outputCol="author1_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="author1_indexed", outputCol="author1_feature"),
        
        StringIndexer(inputCol="author2", outputCol="author2_indexed", handleInvalid='keep'),        
        OneHotEncoder(inputCol="author2_indexed", outputCol="author2_feature"),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        VectorAssembler(inputCols=["content_features", "word_features", "author1_feature", "author2_feature"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [ ]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(2000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_pca.take(1))
df_test_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_pca.take(1))
run_models(df_train_pca, df_test_pca)

**********Run Models with PCA Features**********
[Row(label=1, features=SparseVector(4756, {0: -1.7782, 1: 9.3616, 2: 2.8335, 3: 0.023, 4: -0.0543, 5: -4.8647, 6: 0.692, 7: -0.3924, 8: 2.2732, 9: -2.7471, 10: -0.408, 11: -1.9089, 12: -0.2284, 13: 0.0612, 14: 2.0305, 15: 0.3986, 16: 1.3213, 17: 0.6804, 18: 1.9046, 19: -1.4687, 20: -1.7719, 21: 1.4647, 22: 0.8811, 23: -4.2677, 24: 2.3876, 25: 0.551, 26: 0.9296, 27: 0.1785, 28: 1.4777, 29: 0.7244, 30: -3.1253, 31: -1.1222, 32: 0.2349, 33: 1.643, 34: 0.2642, 35: -0.0612, 36: -1.5604, 37: -0.3543, 38: -0.9313, 39: -1.17, 40: -2.2377, 41: -0.2121, 42: -0.0275, 43: -0.3359, 44: 1.8031, 45: 2.026, 46: 1.1528, 47: 0.4522, 48: 1.8751, 49: 0.6921, 50: 1.6989, 51: 3.7284, 52: 0.1599, 53: -0.7542, 54: -3.0221, 55: 0.4517, 56: 2.9104, 57: 1.1204, 58: -0.0651, 59: -0.3746, 60: -0.1126, 61: -0.845, 62: -0.8866, 63: 1.4559, 64: -0.0799, 65: 0.9001, 66: -0.2304, 67: 1.331, 68: -1.0528, 69: 0.513, 70: 1.7064, 71: 1.8573, 72: -1.2749, 73: 0.5488, 74: -1.0

In [ ]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(2000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_wo_pca.take(1))
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_wo_pca.take(1))

nb_model = NaiveBayes(featuresCol='features', 
                      labelCol='label', 
                      predictionCol='prediction', 
                      probabilityCol='probability', 
                      rawPredictionCol='rawPrediction', 
                      smoothing=1, 
                      modelType='multinomial')

nb_model = nb_model.fit(df_train_wo_pca)
eval_model_perf(nb_model, df_test_wo_pca)

**********Run Models without PCA Features**********
[Row(label=1, features=SparseVector(4464, {0: 3213.0, 1: 2548.0, 2: 73.0, 3: 0.0286, 4: 304.0, 5: 243.0, 6: 0.7993, 2852: 1.0}))]
[Row(label=1, features=SparseVector(4464, {0: 17072.0, 1: 13663.0, 2: 565.0, 3: 0.0414, 4: 1545.0, 5: 933.0, 6: 0.6039, 2022: 1.0}))]
accuracy  =  0.9469114363681469
f1  =  0.9456319370176287
AUC = 0.9821325481619397
CPU times: user 334 ms, sys: 48.1 ms, total: 382 ms
Wall time: 3min 37s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]